# [Memory](https://langchain-ai.github.io/langgraph/concepts/memory/)
- LangGraph의 Memory는 AI가 대화 내용을 기억할 수 있게 해주는 기능입니다. 
- 사람이 대화할 때 이전에 말한 내용을 기억하는 것처럼, AI도 과거 대화를 기억해서 더 자연스러운 대화를 할 수 있게 해줍니다.


## 왜 Memory가 필요한가요?
> Memory가 없다면:
```text
    사용자: "내 이름은 김철수야"
    AI: "안녕하세요!"

    사용자: "내 이름이 뭐였지?"
    AI: "죄송해요, 모르겠습니다"
```
> Memory가 있다면:
```text
    사용자: "내 이름은 김철수야"
    AI: "안녕하세요 김철수님!"

    사용자: "내 이름이 뭐였지?"
    AI: "김철수님이라고 하셨어요!"
```


# LangGraph Memory 예시들

처음 배우는 학생들을 위한 단계별 예시를 준비했습니다!


In [1]:
# 필요한 라이브러리 import
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

True

## 1. Memory가 적용되지 않은 LangGraph 예시
**이름을 기억하지 못하는 챗봇**

### 1. AI 모델 설정

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 2. 그래프 노드 정의 (Memory 없음)

In [3]:
from langgraph.graph import MessagesState

def chat_node(state: MessagesState):
    # 현재 메시지만 처리 (이전 대화 기억 안함)
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

### 3. 그래프 생성

In [4]:
from langgraph.graph import StateGraph, MessagesState, START, END

graph = StateGraph(MessagesState)
graph.add_node("chat", chat_node)
graph.add_edge(START, "chat")
graph.add_edge("chat", END)

### 4. 그래프 컴파일 (Memory 없음)

In [5]:
app_without_memory = graph.compile()

print("Memory 없는 챗봇이 준비되었습니다!")

Memory 없는 챗봇이 준비되었습니다!


### 5. 테스트 

In [6]:
from langchain_core.messages import HumanMessage

# 첫 번째 대화
print("사용자: '내 이름은 김철수야'")
result1 = app_without_memory.invoke({"messages": [HumanMessage(content="내 이름은 김철수야")]})
print(f"AI: {result1['messages'][-1].content}")


사용자: '내 이름은 김철수야'
AI: 안녕하세요, 김철수님! 어떻게 도와드릴까요?


In [7]:
# 두 번째 대화 (새로운 대화)
print("사용자: '내 이름이 뭐였지?'")
result2 = app_without_memory.invoke({"messages": [HumanMessage(content="내 이름이 뭐였지?")]})
print(f"AI: {result2['messages'][-1].content}")

print("\n결과: AI가 이전 대화를 기억하지 못해요!")

사용자: '내 이름이 뭐였지?'
AI: 죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 당신의 이름을 알려주시면 그에 맞춰 대화할 수 있습니다!

결과: AI가 이전 대화를 기억하지 못해요!


## 2. Memory가 적용된 LangGraph 예시 (MemorySaver)
**이름을 기억하는 똑똑한 챗봇**


### 1. AI 모델 설정

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 2. 그래프 노드 정의 (Memory 적용)

In [9]:
from langgraph.graph import MessagesState

def chat_node_with_memory(state: MessagesState):
    # 모든 이전 메시지들과 함께 처리 (Memory가 자동으로 관리)
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

### 3. 그래프 생성

In [10]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver

graph_with_memory = StateGraph(MessagesState)
graph_with_memory.add_node("chat", chat_node_with_memory)
graph_with_memory.add_edge(START, "chat")
graph_with_memory.add_edge("chat", END)

### 4. Memory 저장소 설정 (여기가 핵심!)

In [11]:
memory = MemorySaver()

app_with_memory = graph_with_memory.compile(checkpointer=memory)

print("Memory가 적용된 챗봇이 준비되었습니다!")

Memory가 적용된 챗봇이 준비되었습니다!


### 5. 테스트 

In [12]:
from langchain_core.messages import HumanMessage
import uuid

# 중요: 같은 thread_id를 사용해야 대화가 연결됩니다!
memory_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": memory_id}}

print("=== Memory가 적용된 챗봇 테스트 ===")
print(f"config: {config}")

=== Memory가 적용된 챗봇 테스트 ===
config: {'configurable': {'thread_id': 'dfb15440-6dc3-48f5-9f93-5bd2b492205e'}}


In [13]:
# 첫 번째 대화
print("사용자: '내 이름은 김철수야'")
result1 = app_with_memory.invoke(
    {"messages": [HumanMessage(content="내 이름은 김철수야")]}, 
    config=config  # 여기가 중요!
)
print(f"AI: {result1['messages'][-1].content}")

사용자: '내 이름은 김철수야'
AI: 안녕하세요, 김철수님! 어떻게 도와드릴까요?


In [14]:
# 두 번째 대화 (같은 thread_id 사용)
print("사용자: '내 이름이 뭐였지?'")
result2 = app_with_memory.invoke(
    {"messages": [HumanMessage(content="내 이름이 뭐였지?")]}, 
    config=config  # 같은 config 사용!
)
print(f"AI: {result2['messages'][-1].content}")

print("\n결과: AI가 이전 대화를 기억합니다!")

사용자: '내 이름이 뭐였지?'
AI: 당신의 이름은 김철수입니다. 다른 질문이 있으신가요?

결과: AI가 이전 대화를 기억합니다!
